In [ ]:
!pip install ultralytics

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from Image_Segmentation import *
from itertools import product

In [ ]:
!unzip ndvi_seg_mask_thresh_10.zip

In [ ]:
!rm -rf ndvi_seg_comparison_th_10.zip ndvi_seg_comparison_th_10

In [ ]:
segmentation_model_weights_path = 'yolo_segmentation_model.pt'
segmentation_model = load_yolo_model(segmentation_model_weights_path)
device = 'cpu'

def tile(image, d=2):

  w, h = image.size
  grid = product(range(0, h-h%d, d), range(0, w-w%d, d))
  boxes = []

  for i, j in grid:
      box = (j, i, j+d, i+d)
      boxes.append(box)

  return boxes

In [ ]:
def get_comparison(folder_path, output_folder):

  if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    os.makedirs(os.path.join(output_folder,'crf'))
    os.makedirs(os.path.join(output_folder,'crf_filter'))

  images_list = os.listdir(folder_path)
  for image_name in images_list:
    image_path = os.path.join(folder_path, image_name)
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    original_image = image[:,:512,:].copy()
    ndvi_seg_mask = image[:,512:,:1].copy()/255
    ndvi_seg_mask = ndvi_seg_mask[:,:,0]

    results = segmentation_model.predict(original_image, conf = 0.128, device = device)
    result = results[0]
    if result.masks.data.shape[0] > 4:
      result.masks.data = result.masks.data[:4]
    mask = preprocess_mask(result.masks.data)
    binary_mask_np = generate_binary_mask(mask)
    yolo_seg_image = overlay_mask_on_image(binary_mask_np, original_image)

    refined_mask = ndvi_seg_mask.astype(np.uint8) * 255
    contours, _ = cv2.findContours(refined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    mask = np.zeros(refined_mask.shape, dtype=np.uint8)
    min_area = 500
    for contour in contours:
        if cv2.contourArea(contour) > min_area:
            mask = cv2.drawContours(mask, [contour], -1, 255, -1)
    filtered_mask = cv2.bitwise_and(refined_mask, refined_mask, mask=mask)
    binary_mask_np = generate_binary_mask(filtered_mask)
    ndvi_seg_image = overlay_mask_on_image(binary_mask_np, original_image)
    comparison_image = np.concatenate((original_image, yolo_seg_image , ndvi_seg_image), axis=1)
    output_image_path = os.path.join(output_folder, image_name)
    cv2.imwrite(output_image_path, cv2.cvtColor(comparison_image, cv2.COLOR_BGR2RGB))
    print(f"Image {image_name} saved to {output_image_path}")

In [ ]:
output_folder = 'ndvi_seg_comparison_th_10'
comparison_image = get_comparison('thresh_10', output_folder)

In [ ]:
!zip -r ndvi_seg_comparison_th_10.zip ndvi_seg_comparison_th_10